# =============================================================================
# ANÁLISE PREDITIVA DO TITANIC - PIPELINE COMPLETO
# Este script implementa uma análise completa dos dados do Titanic, incluindo:
# - Pré-processamento e engenharia de features
# - Classificação supervisionada (Random Forest e Decision Tree)
# - Clusterização (KMeans)
# - Análise de regras de associação
# =============================================================================

# 1. IMPORTAÇÕES E CARREGAMENTO
# Bibliotecas para manipulação de dados e visualização

In [ ]:
# =============================================================================
# ANÁLISE PREDITIVA DO TITANIC - PIPELINE COMPLETO
# Este script implementa uma análise completa dos dados do Titanic, incluindo:
# - Pré-processamento e engenharia de features
# - Classificação supervisionada (Random Forest e Decision Tree)
# - Clusterização (KMeans)
# - Análise de regras de associação
# =============================================================================

# 1. IMPORTAÇÕES E CARREGAMENTO
# Bibliotecas para manipulação de dados e visualização
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 1.1 Carregamento dos dados
# Carrega os datasets de treino e teste
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
2. ANÁLISE E PRÉ-PROCESSAMENTO

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
# 2.1 Engenharia de Features
# Extrai o título do nome do passageiro (Mr., Mrs., etc)
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Calcula o tamanho da família (incluindo o próprio passageiro)
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
# Cria uma feature binária indicando se o passageiro está sozinho
train['IsAlone'] = (train['FamilySize'] == 1).astype(int)

# 2.2 Tratamento de Valores Nulos
# Preenche valores nulos com estatísticas apropriadas
train['Age'].fillna(train['Age'].median(), inplace=True)  # Idade com mediana
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)  # Porto com moda
train['Fare'].fillna(train['Fare'].median(), inplace=True)  # Tarifa com mediana

# 2.3 Codificação de Variáveis Categóricas
# Converte variáveis categóricas em numéricas usando Label Encoding
label = LabelEncoder()
train['Sex'] = label.fit_transform(train['Sex'])
train['Embarked'] = label.fit_transform(train['Embarked'])
train['Title'] = label.fit_transform(train['Title'])

# 2.4 Seleção e Preparação das Features
# Define as features que serão utilizadas no modelo
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone', 'Title']
X = train[features]
y = train['Survived']

# 2.5 Padronização dos Dados
# Normaliza as features para terem média 0 e desvio padrão 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
3. CLASSIFICAÇÃO SUPERVISIONADA

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
# 3.1 Divisão dos Dados
# Separa os dados em conjuntos de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3.2 Treinamento e Avaliação dos Modelos
# Random Forest Classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)

# Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_val)

# 3.3 Avaliação dos Resultados
print("Random Forest:\n", classification_report(y_val, y_pred_rf))
print("Decision Tree:\n", classification_report(y_val, y_pred_dt))

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
4. CLUSTERIZAÇÃO

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
# 4.1 Aplicação do KMeans
# Agrupa os passageiros em 3 clusters
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(X_scaled)
clusters = kmeans.labels_

# 4.2 Redução de Dimensionalidade com PCA
# Reduz as features para 2 dimensões para visualização
pca = PCA(n_components=2)
pca_data = pca.fit_transform(X_scaled)

# 4.3 Visualização dos Clusters
plt.figure(figsize=(8,6))
plt.scatter(pca_data[:,0], pca_data[:,1], c=clusters, cmap='viridis')
plt.title('Clusters de Passageiros (KMeans + PCA)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
5. REGRAS DE ASSOCIAÇÃO

## 2. ANÁLISE E PRÉ-PROCESSAMENTO

In [ ]:
# 5.1 Preparação dos Dados
# Seleciona e prepara as features para análise de associação
assoc_data = train[['Sex', 'Pclass', 'Embarked', 'Survived']].copy()
assoc_data['Sex'] = assoc_data['Sex'].map({1: 'male', 0: 'female'})
assoc_data['Survived'] = assoc_data['Survived'].map({1: 'survived', 0: 'died'})

# 5.2 Aplicação do Algoritmo Apriori
# Converte os dados para o formato necessário
records = assoc_data.astype(str).values.tolist()
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df_assoc = pd.DataFrame(te_ary, columns=te.columns_)

# 5.3 Geração e Visualização das Regras
# Encontra padrões frequentes e gera regras de associação
frequent = apriori(df_assoc, min_support=0.1, use_colnames=True)
rules = association_rules(frequent, metric="lift", min_threshold=1.0)

# Exibe as 3 regras mais relevantes
print(rules[['antecedents','consequents','support','confidence','lift']].head(3))